# このノートブックの概要

- ストックマーク社の BERT 本4章を動作確認。

TODO
- hugging face にある他の日本語モデルを調査。
- 複数の文を tokenizer で符号化したあと、一括で復号する方法。（使うことない？）

In [1]:
### fugashi は 形態素解析ツール Mecab を Python から使えるようにしたもの
### ipadic は Mecab で形態素解析を利用するときに使う辞書
!pip install -q transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

In [2]:
### グローバル変数
JAPANESE_WIKI_MODEL = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [3]:
### ライブラリのインポート
import pandas as pd
import torch
from transformers import BertJapaneseTokenizer, BertModel

In [4]:
### トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(JAPANESE_WIKI_MODEL)

### 日本語モデルの語彙数
print(len(tokenizer.vocab))

### id とトークン
df_token = pd.DataFrame([(v, k) for k, v in tokenizer.vocab.items()], columns=['id', 'token'])
df_token

32000


,id,token
0,0,[PAD]
1,1,[UNK]
2,2,[CLS]
3,3,[SEP]
4,4,[MASK]
...,...,...
31995,31995,##з
31996,31996,##侃
31997,31997,##嬪
31998,31998,##崋


In [5]:
text = '''
第一条　この法律は、高度情報通信社会の進展に伴い個人情報の利用が著しく拡大してい
ることに鑑み、個人情報の適正な取扱いに関し、基本理念及び政府による基本方針の作成
その他の個人情報の保護に関する施策の基本となる事項を定め、国及び地方公共団体の責
務等を明らかにするとともに、個人情報を取り扱う事業者の遵守すべき義務等を定めるこ
とにより、個人情報の適正かつ効果的な活用が新たな産業の創出並びに活力ある経済社会
及び豊かな国民生活の実現に資するものであることその他の個人情報の有用性に配慮しつ
つ、個人の権利利益を保護することを目的とする。
'''

### トークン化
input_tokens = tokenizer.tokenize(text)
print(len(input_tokens))
print(input_tokens)

### 符号化
### 符号化のときは先頭に [CLS], 末尾に [SEP] の特殊トークンとして加えるので、
### トークン化したときの数よりも2個多い
input_ids = tokenizer.encode(text)
print(len(input_ids))
print(input_ids)

### 復号
output_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(len(output_tokens))
print(output_tokens)

157
['第', '一', '条', 'この', '法律', 'は', '、', '高度', '情報', '通信', '社会', 'の', '進展', 'に', '伴い', '個人', '情報', 'の', '利用', 'が', '著しく', '拡大', 'し', 'て', 'い', 'る', 'こと', 'に', '鑑み', '、', '個人', '情報', 'の', '適正', 'な', '取扱い', 'に関し', '、', '基本', '理念', '及び', '政府', 'による', '基本', '方針', 'の', '作成', 'その他', 'の', '個人', '情報', 'の', '保護', 'に関する', '施策', 'の', '基本', 'と', 'なる', '事項', 'を', '定め', '、', '国', '及び', '地方', '公共', '団体', 'の', '責', '務', '等', 'を', '明らか', 'に', 'する', 'とともに', '、', '個人', '情報', 'を', '取り扱う', '事業', '者', 'の', '遵守', 'す', 'べき', '義務', '等', 'を', '定める', 'こ', 'と', 'により', '、', '個人', '情報', 'の', '適正', 'かつ', '効果', '的', 'な', '活用', 'が', '新た', 'な', '産業', 'の', '創出', '並びに', '活', '##力', 'ある', '経済', '社会', '及び', '豊か', 'な', '国民', '生活', 'の', '実現', 'に', '資', '##する', 'もの', 'で', 'ある', 'こと', 'その他', 'の', '個人', '情報', 'の', '有用', '性', 'に', '配慮', 'し', 'つ', 'つ', '、', '個人', 'の', '権利', '利益', 'を', '保護', 'する', 'こと', 'を', '目的', 'と', 'する', '。']
159
[2, 97, 52, 631, 70, 2409, 9, 6, 3740, 933, 2035, 745, 5, 10244, 7, 1775, 1963, 933, 5, 666, 14, 

In [6]:
### 元文と符号化復元したテキストの違い
### 元文には改行・空白が含まれていたが、符号化復元した文ではそれらが除かれ、特殊トークンが追加されている
print(text)
print(''.join(output_tokens))


第一条　この法律は、高度情報通信社会の進展に伴い個人情報の利用が著しく拡大してい
ることに鑑み、個人情報の適正な取扱いに関し、基本理念及び政府による基本方針の作成
その他の個人情報の保護に関する施策の基本となる事項を定め、国及び地方公共団体の責
務等を明らかにするとともに、個人情報を取り扱う事業者の遵守すべき義務等を定めるこ
とにより、個人情報の適正かつ効果的な活用が新たな産業の創出並びに活力ある経済社会
及び豊かな国民生活の実現に資するものであることその他の個人情報の有用性に配慮しつ
つ、個人の権利利益を保護することを目的とする。

[CLS]第一条この法律は、高度情報通信社会の進展に伴い個人情報の利用が著しく拡大していることに鑑み、個人情報の適正な取扱いに関し、基本理念及び政府による基本方針の作成その他の個人情報の保護に関する施策の基本となる事項を定め、国及び地方公共団体の責務等を明らかにするとともに、個人情報を取り扱う事業者の遵守すべき義務等を定めることにより、個人情報の適正かつ効果的な活用が新たな産業の創出並びに活##力ある経済社会及び豊かな国民生活の実現に資##するものであることその他の個人情報の有用性に配慮しつつ、個人の権利利益を保護することを目的とする。[SEP]


In [7]:
text = '日本の夜明けぜよ'

### 複数の文をまとめて処理するときの対応：トークン列の長さを揃えるために
### max_length の指定、padding と trunction を以下のように指定
encoding = tokenizer(text,
                     max_length=12,
                     padding='max_length',
                     truncation=True
)

### 符号化した結果
### token_type_ids は文のペアを入力するときに意味を持つ
### attention_mask は [PAD] の位置は0, それ以外は1
print(encoding.keys())
print(encoding['input_ids'])
print(encoding['token_type_ids'])
print(encoding['attention_mask'])

### 復号
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[2, 91, 5, 18924, 4403, 54, 3, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
['[CLS]', '日本', 'の', '夜明け', 'ぜ', 'よ', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [8]:
texts = ['日本の夜明けぜよ', '日本を今一度　せんたくいたし申候']

### 複数の文を処理
encoding = tokenizer(texts,
                     max_length=12,
                     padding='max_length',
                     truncation=True,
                     return_tensors='pt'
)

### GPU 利用
encoding = { k: v.cuda() for k, v in encoding.items() }

encoding

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'),
 'input_ids': tensor([[    2,    91,     5, 18924,  4403,    54,     3,     0,     0,     0,
              0,     0],
         [    2,    91,    11,   744,    52,   559,   191,  1058,  5034,  6785,
          28454,     3]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')}

In [9]:
### モデルのロード
bert = BertModel.from_pretrained(JAPANESE_WIKI_MODEL)

### GPU
bert = bert.cuda()

print(bert.config)

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



- `num_hidden_layers`: レイヤー数は12
- `hidden_size`: BERT の出力は768次元
- `max_position_embedding`: 最大で入力できるトークン列の長さは512

In [10]:
### BERT 処理
output = bert(**encoding)
last_hidden_state = output.last_hidden_state

### 最終層のサイズ、数値
### (2, 12, 768) は、2つの文、12のトークン、768次元の出力を意味する
print(last_hidden_state.shape)
print(last_hidden_state)

### 計算の途中経過を保存しないための処理
with torch.no_grad():
    output = bert(**encoding)
    last_hidden_state = output.last_hidden_state

### キャスト
last_hidden_state = last_hidden_state.cpu()     ### CPU に移す
last_hidden_state = last_hidden_state.numpy()   ### numpy.ndarray
last_hidden_state = last_hidden_state.tolist()  ### リスト

torch.Size([2, 12, 768])
tensor([[[-0.1492, -0.4452,  0.2200,  ...,  0.2751, -0.1948,  0.2257],
         [ 0.1318,  0.3195,  0.1849,  ...,  0.3954, -0.4412, -0.1549],
         [ 0.0778,  0.1387, -0.1841,  ..., -0.0641,  0.2463,  0.2576],
         ...,
         [ 0.0120, -0.0968,  0.0148,  ...,  0.2821, -0.0560, -0.0110],
         [ 0.0280,  0.0099, -0.2381,  ...,  0.2420,  0.0107,  0.2702],
         [ 0.0386, -0.1014,  0.0201,  ...,  0.1868, -0.1122,  0.1650]],

        [[-0.4858,  0.3056, -0.2396,  ..., -0.0197, -0.3537, -0.0568],
         [-0.1149,  0.8836, -0.2248,  ...,  0.1479, -0.1881, -0.6786],
         [ 0.0990,  0.1932, -0.2130,  ...,  0.1664,  0.1663, -0.1990],
         ...,
         [-0.6219, -0.0259,  0.2272,  ..., -0.1069,  0.4126, -0.4511],
         [-0.9980,  0.1628,  0.5400,  ..., -0.5481, -0.0347,  0.1525],
         [-0.8465, -0.0988, -0.6838,  ..., -0.8776,  0.4441,  0.1302]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)
